# Exploring how Lichess' players spend their playing time

*Chess addiction has always been a popular topic, and although its formal existence should be examined through rigorous studies, it has made me curious about how players typically spend their time on Lichess.*

The data was aggregated from the January 2023 Lichess rated games (freely available on the [Lichess database website](https://database.lichess.org)). Bots and games shorter than 4 plies were filtered out. For each account (which from now on will consider equivalent to user), I kept their username, as well as **for each time control** (TC for short, also sometimes referred as category):

*   Their number of games
*   Their average rating over the month
*   The "Estimated time spent", which is the total time they spent playing this TC, with each game duration computed only using the clocks configuration, [using Lichess' formula](https://lichess.org/faq#time-controls) to differenciate between blitz/bullet/etc: (**clock initial time in seconds) + 40 × (clock increment)**.
*   The "Real time spent",  which is the total time they spent playing this TC, based on the time left on the clock at the end of the game, and taking into account increment and berserk. Apart from obvious cases, it does not take into account the use of the `+15s` button, which was considered negligible.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
import statsmodels.api as sm

from pathlib import Path

# Define global style of the plt for all graphs
# /!\ Note that using BOTH style is important (in this order)
plt.style.use("seaborn-v0_8-dark")
plt.style.use("dark_background")
plt.rcParams['axes.facecolor'] = "#444445" # "#444445" # darker is "#262421"
plt.rcParams["figure.facecolor"] = "#333" #"#333" # "#darker is "#161612"
plt.rcParams["grid.color"] = "#fff"
plt.rcParams["grid.linestyle"] = "-."
plt.rcParams["grid.linewidth"] = 0.2
plt.rcParams["axes.grid"] = True
plt.rcParams["savefig.dpi"] = 400
plt.rcParams["savefig.facecolor"] = "auto"
DFT_HEIGHT = 4.8 # inches
DFT_WIDTH = 6.4 # inches

perfs = ["ultrabullet", "bullet", "blitz", "rapid", "classical"]
COLOR_MAP = { "ultrabullet": "#fdb462", "bullet": "#81b1d2", "blitz": "#fa8174", "rapid":"#b3de69", "classical":"#bfbbd9"}
if not __debug__:
    Path("./figures/titles").mkdir(parents=True, exist_ok=True)

FIGURE_NAMES = set()
def figure(save_as_title = False):
    def inner(f):
        def wrapper(*args, **kwargs):
            assert re.match(r"figure_\d+", f.__name__), "figures function should be of the form `figure_<number>`"
            if not __debug__:
                assert not f in FIGURE_NAMES, "figures should not overshadow each other"
                print(f"generating {f.__name__}, save_as_title={save_as_title}")
            FIGURE_NAMES.add(f.__name__)
            fig = f(*args, **kwargs)
            assert isinstance(fig, plt.Figure), "figures function should return a `Figure` object"
            if not __debug__:
                fig.savefig(f"./figures/{f.__name__}.png")
                if save_as_title:
                    print(f"generating title {f.__name__}")
                    axes = fig.get_axes()
                    assert len(axes) == 1, "title is only supported for figure with one subplot"
                    ax = axes[0]
                    ax.set_xticks([])
                    ax.set_yticks([])
                    fig.savefig(f"./figures/titles/{f.__name__}.png", bbox_inches='tight',pad_inches = 0)
        return wrapper
    return inner

In [ ]:
dtypes = {"username": "string"}

#for perf in perfs:
#    dtypes[f"{perf}_approximate_time"] = "Int64"
#    dtypes[f"{perf}_real_time"] = "Int64"
#    dtypes[f"{perf}_avg_rating"] = "Int64"

def restrict_to_perf(perf: str, with_error = True):
    l = ["username", f"{perf}_games", f"{perf}_avg_rating",f"{perf}_approximate_time", f"{perf}_real_time"]
    if with_error:
        l.append(f"{perf}_relative_error")
    return l

#df = pd.read_csv("time-spent.csv",dtype=dtypes)
df = pd.read_csv("time-spent-2023-01.csv",dtype=dtypes)
#df.dtypes

In [ ]:
#df_time = df.copy()

def convert(c):
    df_time[c] = df_time[c].apply(lambda x: pd.to_timedelta(x,unit="S"))

#for perf in perfs:
#    convert(f"{perf}_approximate_time")
#    convert(f"{perf}_real_time")
#df_time

Here is an extract from the data for the bullet category.

In [ ]:
#table_df = df_time[df_time["bullet_games"] > 2].iloc[:5][restrict_to_perf("bullet",False)]
#table_df.rename(columns={c:  c.replace("bullet_","") for c in restrict_to_perf("bullet",False)},
#          inplace=True, errors='raise')

#print(table_df.to_markdown())

|    | username            |   games |   avg_rating | approximate_time   | real_time       |
|---:|:--------------------|--------:|-------------:|:-------------------|:----------------|
|  0 | ange_de_la_mode     |      85 |         1430 | 0 days 01:32:20    | 0 days 02:26:59 |
|  5 | Ankit_khandelwal_07 |     214 |         1316 | 0 days 03:39:00    | 0 days 05:33:30 |
|  8 | jpn26               |     550 |         1577 | 0 days 09:10:00    | 0 days 14:07:11 |
| 10 | ptitbill            |     303 |         1465 | 0 days 13:28:00    | 0 days 20:37:07 |
| 17 | Inkety              |      31 |         1467 | 0 days 00:31:00    | 0 days 00:42:31 |

They were ~~2,073,933~~ 2,073,569 (removing bots) distinct users who played that month!

In [ ]:
#df_time.describe()

## How is time spent playing distributed by time control?

In [ ]:
def sum_per_perf_of_col(df, column: str):
    return df[[column.format(p) for p in perfs]].sum()

def count_per_perf_of_col(df, column: str):
    return df[[column.format(p) for p in perfs]].count()

def func(pct):
    if pct < 1:
        return
    #absolute = int(np.round(pct/100.*np.sum(allvals)))
    return "%d%%" % pct

@figure()
def figure_1():
    players = count_per_perf_of_col(df, "{}_games")
    games = sum_per_perf_of_col(df,"{}_games")
    real_time = sum_per_perf_of_col(df,"{}_real_time")
    approx_time = sum_per_perf_of_col(df,"{}_approximate_time")
    datas = [{
        "title": "Players",
        "data": players,
        "x_axis": "number of players",
    },{
        "title": "Games",
        "data": games,
        "start_from_right": True,
        "x_axis": "number of games",
    },{
        "title": "Total real time played",
        "data": real_time,
        "x_axis": "real time played (s)",
    },{
        "title": "Total estimated time played",
        "data": approx_time,
        "start_from_right": True,
        "x_axis": "estimated time played (s)",
    }]

    factor = 1.5
    fig, axs = plt.subplots(ncols=2,nrows=2, figsize=(DFT_WIDTH * factor, DFT_HEIGHT * factor))
    fig.subplots_adjust(hspace=.3)
    axs = axs.flatten()
    for ax, data in zip(axs, datas):
        for col_name, col_values in data["data"].sort_values(ascending=False).items():
            perf = col_name.split("_")[0]
            rects = ax.barh(perf,width=col_values, color=COLOR_MAP[perf], height=.6)
            ax.invert_yaxis()
            ax.set_xlabel(data["x_axis"])
            if data.get("start_from_right"):
                ax.invert_xaxis()
                ax.yaxis.set_label_position("right")
                ax.yaxis.tick_right()
        ax.set_title(f"{data['title']} per time control")
    fig.suptitle("Number of players, games and time played by time control",y=.94)
    return fig

figure_1()

*Figure 1*

As expected the number of games alone is not a good appreciation of how much total time is spent playing.
On the other hand, we can observe that the estimated time formula is fairly accurate for all but the classical category, where the total estimated time played is much higher than real time spent.

## In which time control do individual players spend the most time?

In [ ]:
def add_lichess_knight_bg(ax):
    datafile = 'user-blog-default.png'
    img = plt.imread(datafile)
    xl, xh = ax.get_xlim()
    yl, yh = ax.get_ylim()
    ax.imshow(img, zorder=0,alpha=0.5,origin="upper",extent=[xl,xh, yl, yh],aspect="auto")

def real_time_box_plot(ax, df, perfs):
    data = df[[f"{x}_real_time" for x in perfs]]
    data_dropna = [values.dropna() for (_, values) in data.items()]
    colors = [COLOR_MAP[x] for x in perfs]
    bplot = plt.boxplot(data_dropna,patch_artist=True,sym='',labels=perfs,showmeans=True,meanprops={ "markerfacecolor":"white", "markeredgecolor":"white"})
    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)

@figure(save_as_title=True)
def figure_2():
    fig = plt.figure(figsize=(DFT_WIDTH * 8 / 5, DFT_WIDTH))
    ax = fig.add_subplot(1,1,1,)#nrows=1,)
    real_time_box_plot(ax,df,perfs)
    add_lichess_knight_bg(ax)
    ax.set_ylabel("real time spent playing in that time control (s)")
    fig.suptitle("Real time spent playing distribution by time control")
    fig.set_tight_layout(True) # for the blog post head image
    return fig

figure_2()

*Figure 2: For each time control only players with at least one game in that TC were included. Whiskers display 1.5x IQR, outliers are hidden.*

The graph demonstrates that the significant popularity of blitz is not only due to the fact that there are more players compared to bullet or rapid, but also because, on average (shown by the triangle in the figure) and median, blitz players spend more time playing it than other players play in other time controls. Another point to consider regarding the average time spent playing is that it frequently exceeds the 75th percentile, indicating the presence of significant outliers (extremely active players in our case).

## Are active players versatile or specialised?

Let's first see if players play multiple time controls at all.

In [ ]:
from PIL import Image, ImageOps
from dataclasses import dataclass

# https://stackoverflow.com/a/60737369/11955835
def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, int(delta_width - pad_width), delta_height - pad_height)
    return ImageOps.expand(img, padding)

# Make it so all images have 1.33 aspect ratio
def normalise_img(path):
    desired_aspect_ratio = 1.33
    old = Image.open(path)
    width, height = old.size
    current_asp = width / height
    if abs(current_asp - desired_aspect_ratio) < 0.01:
        return old # Good enough
    return resize_with_padding(old, (int(width * desired_aspect_ratio), int(width)))

VALUE_TO_PERF_ICON = {(i * 6): f"perfs-icon/{p}.png" for (i, p) in enumerate(perfs)}

@dataclass
class CorrMatrix:
    v_min: float
    v_max: float
    color_gradient: any
    format: str
    color_label: str
    to_value: any = lambda x: x # function T -> value between v_min and v_max

    def matrix_case(self, fig, ax, index, value):
        ax.set_xticks([])
        ax.set_yticks([])
        if index in VALUE_TO_PERF_ICON.keys():
            img = np.array(normalise_img(VALUE_TO_PERF_ICON[index]))
            ax.imshow(img)
            ax.set_facecolor(fig.get_facecolor())
            return
        norm = mpl.colors.Normalize(vmin=self.v_min, vmax=self.v_max)
        ax.text(0.5, 0.5, self.format.format(value=value), horizontalalignment='center', verticalalignment='center',fontsize=14, weight='bold',color=self.color_label)
        color = self.color_gradient(norm(self.to_value(value)))
        # Create a color gradient based on value
        # Set the background color of the figure
        ax.patch.set_facecolor(color)

    def matrix_layout(self, values, axs, fig):
        for index, (value, ax) in enumerate(zip(values, axs)):
            self.matrix_case(fig=fig, ax=ax, index=index,value=value)
            if index % 5 == 0: # first column
                ax.set_ylabel(perfs[index // 5])
            else:
                ax.set_yticklabels([])
            if index >= 20: # last row
                ax.set_xlabel(perfs[index - 20])
            else:
                ax.set_xticklabels([]) # useful?

    def draw_plot(self, values):
        mother_fig = plt.figure(figsize=(DFT_WIDTH * 1.5, DFT_HEIGHT * 1.5))
        subfigs = mother_fig.subfigures(ncols=2, nrows=1, width_ratios=[15,1])
        matrix_fig = subfigs[0]
        axs = matrix_fig.subplots(ncols=len(perfs),nrows=len(perfs))
        axs = axs.flatten()
        matrix_fig.subplots_adjust(wspace=0, hspace=0)
        
        self.matrix_layout(values=values, axs=axs,fig=matrix_fig)
        
        color_ax = subfigs[1].add_subplot(1,1,1,)
        colorbar = mpl.cm.ScalarMappable(mpl.colors.Normalize(vmin=self.v_min, vmax=self.v_max), cmap=self.color_gradient)
        colorbar_plot = subfigs[1].colorbar(colorbar, cax=color_ax)
        return mother_fig, matrix_fig, colorbar_plot


In [ ]:
@figure()
def figure_3():
    intersect_values = []
    for p1 in perfs:
        for p2 in perfs:
            nb_players_p1 = df[f"{p1}_real_time"].count()
            # players playing p1 AND p2
            nb_players_p1_p2 = df.dropna(subset=[f"{p1}_real_time", f"{p2}_real_time"])[f"{p1}_real_time"].count()
            intersect_values.append(nb_players_p1_p2 / nb_players_p1 * 100)
    
    intersect_matrix = CorrMatrix(v_min=0,v_max=100, color_gradient=plt.cm.viridis,format="{value:.1f}%",color_label="white")
    mother_fig, fig, colorbar_plot = intersect_matrix.draw_plot(intersect_values)
    colorbar_plot.set_label('Percentage of players playing both time controls')
    fig.suptitle("Intersection of players populations per time control",y=0.92,size="x-large")
    fig.supylabel("Players who play", x=0.08)
    fig.supxlabel("also play", y=0.06)
    return mother_fig

figure_3()

*Figure 3: Players of a time control are defined as all players that have played at least one game in that specific TC.*

For example, the bottom-left square (7.4%) indicates that among the players who have played at least one classical game, 7.4% of them have also played at least one ultrabullet game.
The matrix is not symmetrical along its diagonal because it does not compare the same population. Taking the same example, the bottom-left corner represents 7.4% out of **all classical players**, whereas its symmetrical counterpart (top-right corner, 25.9%) refers to all **ultrabullet players**, who are fewer in number (Figure 1).

Overall, there is a relatively low intersection between time controls, with only half of rapid players also playing blitz, despite rapid and blitz being the two most popular time controls. Players playing in less popular time controls also more likely to engage in more mainstream ones, with over 80% of ultrabullet players participating in bullet, and around 70% of classical players also playing rapid.

Now, let's examine whether players who are more active in one time control tend to be more active in the others when they participate in multiple time controls.

In [ ]:
@figure()
def figure_4():
    corr_method = "Spearman"
    
    corr_matrix = CorrMatrix(v_min=-1,v_max=1, color_gradient=plt.cm.bwr,format="{value:.2f}",color_label="black")
    values = df[[f"{perf}_real_time" for perf in perfs]].corr(method=corr_method.lower()).values.flatten()
    mother_fig, fig, colorbar_plot = corr_matrix.draw_plot(values=values)
    colorbar_plot.set_label(f'{corr_method} correlation coefficient')
    fig.suptitle("Correlation matrix of time spent playing by time control",y=0.92,size="x-large")
    label = "real time spent"
    fig.supxlabel(label, y=0.06)
    fig.supylabel(label, x=0.08)
    return mother_fig

figure_4()

*Figure 4: The [Spearman's](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient) correlation coefficient is only calculated among the players who have played at least one game in both relevant time controls.*

The Spearman's correlation coefficient measures the strength of the relationship between spending more time playing in time control `X` and playing more in the time control `Y`.

In general, it seems that players tend to focus on one specific time control. The correlation between ultrabullet and bullet is by far the strongest, followed by rapid and classical. This is quite logical as those time controls are very similar. The correlation between other time controls is significantly lower, and the more different the TCs are the lower it is.

## How is the time spent playing distributed among the players?

Now that we have a first overview of the time spent playing by time controls, it's interesting to consider how in a specific TC the playing time is distributed.

In [ ]:
def hist(df, ax, data, color,perf,log: bool):
    ax.hist(df[data["column"]].astype("float64"),bins=100,color=color,log=log,label=perf,alpha=1)
    ax.set_xlabel(data["xlabel"])
    ax.set_ylabel("number of players")

def perf_hist(df, axs, perf: str,include_avg=False,log=False):
    datas = [{
        "column": f"{perf}_games",
        "xlabel": "number of games",
    }]
    if include_avg:
        datas.append({
       "column": f"{perf}_avg_rating",
       "xlabel": "average rating",
        })
    datas.extend([
    {
        "column": f"{perf}_approximate_time",
        "xlabel": "estimated time (s)",
    },{
        "column": f"{perf}_real_time",
        "xlabel": "real time (s)",
    }])
    for (ax, data) in zip(axs, datas):
        hist(df,ax,data,COLOR_MAP[perf],perf,log=log)

@figure()
def figure_5():
    df1 = df[restrict_to_perf("bullet",False)].copy()
    df1.sort_values("bullet_real_time", ascending=False, inplace=True)
    fig, axs = plt.subplots(ncols=2,nrows=2,figsize=(DFT_WIDTH * 1.7, DFT_HEIGHT * 1.5))
    axs = axs.flatten()
    perf_hist(df1, axs, "bullet",include_avg=True)
    fig.suptitle(f"Bullet distribution of players by games, rating and time played",y=.93)
    return fig

figure_5()

*Figure 5*

The number of players decreases exponentially as the number of games and time played increases, while the rating distribution follows the expected normal distribution. A surprising finding is the significant presence of new players (indicated by the large spike at 1500 rating), in contrast to the [online rating distribution, which only takes into account players with a stable rating](https://lichess.org/stat/rating/distribution/bullet).

Let's see whether this exponential decrease is is also observed in the other time controls, using with a logarithmic y-axis which will be more appropriate.

In [ ]:
@figure()
def figure_6():
    fig, axs = plt.subplots(ncols=3,nrows=len(perfs),figsize=(DFT_WIDTH * 2.5, DFT_HEIGHT * len(perfs) / 1.5))
    
    for perf, axs_row in zip(perfs, axs):
        perf_hist(df, axs_row,perf,log=True)
        axs_row[0].legend()
    fig.suptitle(f"Distribution of players by games and time played, for each time control",y=0.92,size="x-large")
    return fig

figure_6()

*Figure 6*

While this overview clearly demonstrates that they all follow the same distribution pattern, the lack of a common scale makes it difficult to compare them.

In [ ]:
def lowess(df, col_y, col_x, frac):
    #x = np.random.uniform(low = -2*np.pi, high = 2*np.pi, size=500)
    y = df[col_y]
    x = df[col_x]
    lowess = sm.nonparametric.lowess(endog=y, exog=x,frac=frac)
    df[col_x] = lowess[:, 0]
    df[col_y] = lowess[:, 1]
    
def get_distrib(*, df, col,bins, frac: int | None):
    distribs = [np.histogram(df[col.format(perf)].dropna(),bins=bins) for perf in perfs]
    distrib_plot = []
    for distrib in distribs:
        # we want to compute the average of the bucket
        average_x = pd.DataFrame(distrib[1], columns=["bucket"])["bucket"].rolling(2).mean().dropna()
        res = pd.DataFrame(zip(average_x, distrib[0]), columns=['avg_buckets', 'nb_players'])
        # drop points with 0 players
        res = res[res["nb_players"] != 0]
        if frac:
            lowess(res, col_y="nb_players", col_x="avg_buckets",frac=frac)
        distrib_plot.append(res)
    return distrib_plot

def draw_loglog(*, ax, df, col,bins,frac):
    for perf, plot_df in zip(perfs, get_distrib(df=df, col=col,bins=bins,frac=frac)):
        ax.loglog(plot_df["avg_buckets"], plot_df["nb_players"],color=COLOR_MAP[perf],label=perf)
    #ax.set_yscale('log')
    #ax.set_xscale('log')
    ax.set_ylabel("number of players")
    ax.legend()

@figure()
def figure_7():
    fig, ax = plt.subplots(ncols=2,nrows=1, figsize=(DFT_WIDTH * 2, DFT_HEIGHT))
    
    draw_loglog(ax=ax[0], df=df, col="{}_games",bins=100,frac=0.1)
    ax[0].set_title("number of games distribution (LOWESS Smoothing)")
    ax[0].set_xlabel("number of games played")
    
    draw_loglog(ax=ax[1], df=df, col="{}_real_time",bins=10_000,frac=0.011)
    ax[1].set_title("real time spent playing distribution (LOWESS Smoothing)")
    ax[1].set_xlabel("real time spent playing (s)")
    return fig

figure_7()

*Figure 7*

This loglog graph confirms the exponential decrease of players per number of games played. 

Additionally, there is a noticeable spike in real time played for each time control (but ultrabullet), which roughly corresponds to to the expected duration of one game in that specific category. For slower time controls, the spike in game duration is smoother, indicating a greater variety in absolute game duration.

For (ultra)bullet, blitz, and rapid, a longer time control result in a longer game duration. However, this pattern does not hold true for classical time control. We can suppose that either players play faster than they should in classical games, or that classical games last fewer moves on average since it is harder to recover from a losing position.

The games and time spent playing are very unbalanced among players, but let's quantify it.

In [ ]:
# Compute LORENZ graph per `perf`
def cumul_by_biggest_player_first(df, perf):
    col = f"{perf}_real_time"
    sorted = df.sort_values(col, ascending=False)
    # drop=True parameter is used to avoid adding the old index as a new column in the resulting Series
    cumsum = sorted[col].cumsum().reset_index(drop=True).dropna()
    cumsum.name = "cum_sum"
    relative_sorted = pd.DataFrame(cumsum, columns=['cum_sum'])
    all_time_spent = cumsum.iloc[-1]
    nb_players = cumsum.count()
    # 200 is used as debugging value as real should be between 0 and 100
    relative_sorted = relative_sorted.assign(
        ranking=lambda x: x.index / nb_players * 100, 
        relative_all_time_spent=lambda x: x.cum_sum / all_time_spent * 100
    )
    return relative_sorted

@figure(save_as_title=True)
def figure_8():
    factor = 1
    fig = plt.figure(figsize=(DFT_WIDTH * factor * 8 / 5, DFT_WIDTH * factor))
    ax = fig.add_subplot(111)
    x = np.linspace(0, 100, 100)
    
    plt.plot(x, x, color='grey',linestyle="-.")  # Plot the identity line
    for perf in perfs:
        relative_sorted = cumul_by_biggest_player_first(df, perf)
        ax.plot("ranking", "relative_all_time_spent", data=relative_sorted, color=COLOR_MAP[perf],label=perf)
    ax.legend()
    add_lichess_knight_bg(ax)
    ax.set_xlabel("cumulative share of players (%)")
    ax.set_ylabel("cumulative share of total real time spent (%)")
    fig.suptitle("Lorenz curve of real time spent per player", y=.92)
    return fig

figure_8()

*Figure 8*

The [Pareto principle](https://en.wikipedia.org/wiki/Pareto_principle), which states that in many cases "roughly 80% of consequences come from 20% of causes" is roughly respected.

In fact it is even more pronounced in the ultrabullet category, where only about 10% of the players account for 80% of the time played in that time control. We can observe that the more popular a time control is, the more evenly the time played is distributed among players.

Classical chess stands out in this regard, with the top 15% most active players being highly committed, followed by a sharp drop-off of the curve, meaning the time is more evenly spread out between players after that point. This could be explained by the fact that the time commitment required to start a classical game filters out the most casual players compared to bullet or blitz, only leaving the moderately active players. This trend is also shown in Figure 2 where the range between the 25th and 75th percentile of time played is narrower compared to other time controls.

## How accurate is the estimated game duration formula?

Let's now look a bit deeper into the estimated game duration formula used by Lichess.

In [ ]:
def filter_relative_error(df, perfs):
    return [df[df[f"{c}_relative_error"] < df[f"{c}_relative_error"].quantile(0.99)][f"{c}_relative_error"] for c in perfs]

def relative_better_boxplot(ax, df, perfs):
    data = filter_relative_error(df, perfs)
    colors = [COLOR_MAP[x] for x in perfs]
    bplot = ax.boxplot(data,patch_artist=True,sym='',labels=perfs,showmeans=True,meanprops={ "markerfacecolor":"white", "markeredgecolor":"white"})
    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)

@figure()
def figure_9():
    # Compute the relative error of the real time spent, compared to the average computed one
    df_avg_error = df.copy()
    for perf in perfs:
        df_avg_error[f"{perf}_relative_error"] = (df[f"{perf}_approximate_time"] - df[f"{perf}_real_time"]).abs() / df[f"{perf}_real_time"]
    avg_errors = [f"{p}_relative_error" for p in perfs]
    
    fig, axs = plt.subplots(ncols=2,nrows=1,figsize=(DFT_WIDTH * 1.5, DFT_HEIGHT * 1.2),gridspec_kw={"width_ratios": [4, 1]})
    relative_better_boxplot(axs[0],df_avg_error,perfs[:-1])
    y_label = "absolute relative error"
    axs[0].set_ylabel(y_label)
    relative_better_boxplot(axs[1],df_avg_error,perfs[-1:])
    axs[1].set_ylabel(y_label)
    fig.suptitle("Game duration estimation absolute relative error")
    return fig

figure_9()

*Figure 9: Whiskers display 1.5x IQR, outliers are hidden.*

As we can see the formula is fairly accurate for short time controls, but its accuracy decreases for longer TCs, to the point that a separate scale was required for classical.

Now, to determine whether the formula overestimates or underestimates the duration of the game, we need to analyze the distribution of non-absolute relative errors.

In [ ]:
APPROX_ERROR_MIN_QUANTILE = 0.01
APPROX_ERROR_MAX_QUANTILE = 0.95

def perf_hist_err(df, ax, perf: str):
    col = f"{perf}_relative_error"
    data = df[ (df[col] < df[col].quantile(APPROX_ERROR_MAX_QUANTILE)) & (df[col] > df[col].quantile(APPROX_ERROR_MIN_QUANTILE))][col]
    ax.hist(data,bins=100,color=COLOR_MAP[perf],label=perf)
    ax.set_xlabel("relative error")
    ax.set_ylabel("number of players")
    ax.legend()

@figure()
def figure_10():
    non_abs_rel_error = df.copy()
    for perf in perfs:
        non_abs_rel_error[f"{perf}_relative_error"] = (df[f"{perf}_approximate_time"] - df[f"{perf}_real_time"]) / df[f"{perf}_real_time"]
    
    
    fig, axs = plt.subplots(ncols=2,nrows=3,figsize=(DFT_WIDTH * 2, DFT_HEIGHT * 2))
    axs = axs.flatten()
    
    for perf, ax in zip(perfs, axs):
        perf_hist_err(non_abs_rel_error, ax,perf)
    axs[-1].axis('off')
    fig.suptitle(f"Game duration formula relative error (quantiles {APPROX_ERROR_MIN_QUANTILE} to {APPROX_ERROR_MAX_QUANTILE})",y=0.92,size="x-large")
    return fig

figure_10()

*Figure 10*

The formula tends to underestimate the game duration in short time controls, then starts overestimating it. I am not sure what phenomon create these spikes in the ultrabullet distribution, but it is probably due to the limited accuracy of measuring game duration (down to the second).

## Do better players play more?

Although correlation does not imply causation, it is intriguing to investigate whether more skilled players tend to spend more time playing than the average players. A scatter plot of the amount of time spent playing against player ratings is a good first approach to detect patterns or trends.

In [ ]:
# Scatter plot, x rating, y time-spent

def scatter(df, ax, perf):
    ax.scatter(x=df[f"{perf}_avg_rating"],y=df[f"{perf}_real_time"],color=COLOR_MAP[perf],s=1)
    ax.set_ylabel("real time (s)")
    #ax.set_label(perf)
    ax.set_xlabel("rating")

@figure()
def figure_11():
    fig, axs = plt.subplots(ncols=2,nrows=3,figsize=(DFT_WIDTH * 2, DFT_HEIGHT * 2))
    axs = axs.flatten()
    for ax, perf in zip(axs, perfs):
        scatter(df, ax, perf)
        ax.legend(labels=[perf],markerscale=5,markerfirst=False) # FIXME: couldn't get to make the legend box appear 
    # last one is all combined
    for perf in perfs:
        scatter(df, axs[-1], perf)
    axs[-1].legend(labels=perfs,markerscale=5,markerfirst=False) # FIXME: couldn't get to make the legend box appear 
    fig.suptitle("Real time played per rating",y=0.92,size="x-large")
    return fig

figure_11()

*Figure 11*

There seems to be a slight tendency for players above 1500 to play more, we can confirm that by checking the rating distribution weighted by time played.

In [ ]:
# Weighted histogram. Total time per rating

def perf_hist_weighted(df, ax, perf: str):
    ax.hist(df[f"{perf}_avg_rating"],bins=100,color=COLOR_MAP[perf],weights=df[f"{perf}_real_time"],density=True, label=f"weighted {perf}")
    ax.hist(df[f"{perf}_avg_rating"],bins=100,alpha=0.4,density=True,edgecolor = "black",color="white",label=perf) # color=COLOR_MAP[perf]
    ax.set_xlabel("rating")
    ax.set_ylabel("density")
    ax.legend()
    #ax.set_title(perf)

@figure()
def figure_12():
    fig, axs = plt.subplots(ncols=2,nrows=3,figsize=(DFT_WIDTH * 2, DFT_HEIGHT * 2))
    axs = axs.flatten()
    for perf, ax in zip(perfs, axs):
        perf_hist_weighted(df, ax,perf)
    axs[-1].axis('off')
    fig.suptitle("rating density, weighted by time played",y=0.92,size="x-large")
    return fig

figure_12()

*Figure 12*

Proportionally weighting by time played has the consequence of erasing the new players spike at 1500 in all but classical time controls, as well as shifting the bell curve to the right (compared to the unweighted version), confirming that **better players (as in players above the median) play more games**.

## Conclusion

If you feel trapped in an unstoppable blitz frenzy, you are not alone!

In [ ]:
def plot_cum_perfs(ax, df):
    usernames = df["username"]
    time_played_acc = np.zeros(len(usernames))
    for perf in perfs:
        data = df[f"{perf}_real_time"].fillna(0)
        rects = ax.barh(usernames, data, color=COLOR_MAP[perf],left=time_played_acc,label=perf)
        time_played_acc += data
    assert time_played_acc.equals(df["total_real_time"])

    ax.bar_label(rects, df["total_formated"],
                  padding=-100, color='white', fontweight='bold')
    ax.legend()
    ax.invert_yaxis() # to have the top value at the top
    ax.set_xlim(right=time_played_acc.max() * 1.3)

@figure()
def figure_13():
    df_total = df.copy()
    df_total["total_real_time"] = df[[f"{perf}_real_time" for perf in perfs]].sum(axis=1)
    df_total["total_formated"] = df_total["total_real_time"].apply(lambda x: pd.to_timedelta(x,unit="S"))
    df_total.sort_values("total_real_time", ascending=False, inplace=True)    
    fig = plt.figure(figsize=(DFT_WIDTH * 1.5, DFT_HEIGHT * 1.5))
    ax = fig.add_subplot(1,1,1,)
    top = 20
    plot_cum_perfs(ax, df_total.iloc[:top])
    ax.set_title(f"Top {top} most active players")
    ax.set_xlabel("real time (s)")
    return fig

figure_13()

*Figure 13*

*Code can be found [here](https://github.com/kraktus/lichess-time-spent). Thanks to @somethingpretentious and @glbert for their extensive feedback and expertise.*